In [76]:
import warnings
import keplergl
import numpy as np
import osmnx as ox
import pandas as pd
import geopandas as gpd
from sklearn import preprocessing
import plotly.express as px
from skgstat import Variogram
import matplotlib.pyplot as plt
from shapely.geometry import Point
from pykrige.ok import OrdinaryKriging
from scipy.interpolate import NearestNDInterpolator
from tobler.area_weighted import area_interpolate
import plotly.express as px
import json
import ogr
import ipywidgets as widgets
from IPython.display import display

In [5]:
amen1 = pd.read_csv('vancouver_facilities_2.csv')
amen1.head()

,id,lat,lon,type,name,city,city_id
0,10,49.1763542,-123.112783,museum,12 Service Battalion Museum,Richmond,5915015
1,15,49.261938,-123.151123,museum,15th Field Artillery Regiment Museum And Archives,Vancouver,5915022
2,24,49.278786,-123.098796,museum,221A Artist Run Centre,Vancouver,5915022
3,41,49.2210003,-123.0091848,artist,7302754 Canada Inc,Burnaby,5915025
4,97,49.14709735,-122.6467963,heritage or historic site,Abc Heritage Preschool And Child Care,Langley,5915001


In [44]:
amen = pd.read_csv('vancouver_facilities.csv')
amen.head()

,Index,Facility_Name,Source_Facility_Type,ODCAF_Facility_Type,Provider,Unit,Street_No,Street_Name,Postal_Code,City,Prov_Terr,Source_Format_Address,CSD_Name,CSDUID,PRUID,Latitude,Longitude
0,10,12 Service Battalion Museum,military museum or fort,museum,canadian museums association,..,5500,no 4 rd,V6X 3L5,richmond,bc,5500 no. 4 rd the sherman armoury richmond bri...,Richmond,5915015,59,49.1763542,-123.112783
1,15,15th Field Artillery Regiment Museum And Archives,museum/gallery,museum,vancouver,..,2025,11th av w,V6J 2C7,vancouver,bc,2025 w 11th av vancouver bc v6j 2c7,Vancouver,5915022,59,49.261938,-123.151123
2,24,221A Artist Run Centre,museum/gallery,museum,vancouver,..,221,georgia st e,V6A 1Z6,vancouver,bc,221 e georgia st vancouver bc v6a 1z6,Vancouver,5915022,59,49.278786,-123.098796
3,41,7302754 Canada Inc,artist making fine art and exhibiting in galle...,artist,burnaby,..,6909,mckay av,..,burnaby,bc,6909 mckay av,Burnaby,5915025,59,49.2210003,-123.0091848
4,97,Abc Heritage Preschool And Child Care,preschool and out of school care,heritage or historic site,township of langley,..,7949,208 st,..,township of langley,bc,..,Langley,5915001,59,49.14709735,-122.6467963


In [45]:
amen = amen[['Index', 'Latitude', 'Longitude', 'ODCAF_Facility_Type', 'Facility_Name', 'City', 'Postal_Code']]
amen.head()

,Index,Latitude,Longitude,ODCAF_Facility_Type,Facility_Name,City,Postal_Code
0,10,49.1763542,-123.112783,museum,12 Service Battalion Museum,richmond,V6X 3L5
1,15,49.261938,-123.151123,museum,15th Field Artillery Regiment Museum And Archives,vancouver,V6J 2C7
2,24,49.278786,-123.098796,museum,221A Artist Run Centre,vancouver,V6A 1Z6
3,41,49.2210003,-123.0091848,artist,7302754 Canada Inc,burnaby,..
4,97,49.14709735,-122.6467963,heritage or historic site,Abc Heritage Preschool And Child Care,township of langley,..


In [46]:
library = amen[amen['ODCAF_Facility_Type'] == 'library or archives']
library.head()

,Index,Latitude,Longitude,ODCAF_Facility_Type,Facility_Name,City,Postal_Code
6,140,49.243554,-121.992562,library or archives,Agassiz Library,north vancouver,V0M 1A0
9,215,49.055541,-122.481863,library or archives,Aldergrove Library,vancouver,V4W 3B8
16,358,49.2476604,-123.1545938,library or archives,Archives of Anglican Provincial Synod of Briti...,vancouver,V6H 2E2
44,744,52.37201,-126.757864,library or archives,Bella Coola Branch,burnaby,V0T 1C0
45,1749,52.41529,-121.847499,library or archives,Big Lake,vancouver,V0L 1G0


In [55]:
library[["Latitude", "Longitude"]] = library[["Latitude", "Longitude"]].apply(pd.to_numeric, errors='coerce')
library = library.dropna()
library = library.reset_index(drop=True)

In [ ]:
# lat and lon need to be numeric for joining

library = library.reset_index(drop=True)
library["Latitude"] = pd.to_numeric(library["Latitude"])
library["Longitude"] = pd.to_numeric(library["Longitude"])

In [65]:
lib_bc = pd.read_csv('public-library-branch-data.csv')
lib_bc.head()

,LIBRARY_SYSTEM,LOCATION,BRANCH_UNIQUE_ID,SCHOOL_DISTRICT_SERVED,PHONE,PHYSICAL_ADDRESS,CITY,PROVINCE,POSTAL_CODE,LATITUDE,...,LIBRARIAN_HRS_B�,LIB_TECH_HRS_B�,COMM_LIB_HRS_B�,OTH_HRS_B�,branch_copiers�,LEED_CERT_B�,SHRD_FAC�,FLOORSPACE�,HRS_OPEN�,DAYS_OPEN�
0,Alert Bay Public Library & Museum,Alert Bay Public Library & Museum,BABM001,85.0,(250) 974-5721,116 Fir Street,Alert Bay,BC,V0N 1A0,50.584823,...,0,0,622,281,1.0,NaN,NaN,213.36,618,191.0
1,Beaver Valley Public Library,Beaver Valley Public Library,BFBV001,20.0,(250) 367-7114,1847 - 1st Street,Fruitvale,BC,V0G 1L0,49.112966,...,0,0,4940,1350,1.0,No,No,270,2320,321.0
2,Bowen Island Public Library,Bowen Island Public Library,BBI001,45.0,(604) 947-9788,430 Bowen Island Trunk,Bowen Island,BC,V0N 1G0,49.379906,...,1820,0,0,4107,1.0,No,Yes,407,1953,303.0
3,Burnaby Public Library,Bob Prittie Metrotown Branch,BB001,41.0,(604) 436-5427,6100 Willingdon Avenue,Burnaby,BC,V5H 4N5,49.228301,...,"42,224","1,733",NaN,"104,199",4.0,No,No,"5,673.00","3,612",339.0
4,Burnaby Public Library,Cameron Library & Recreation Centre,BB002,41.0,(604) 421-5454,9523 Cameron Street,Burnaby,BC,V3J 1L6,49.253814,...,"6,834",NaN,NaN,"16,311",1.0,No,Yes,465,"2,901",341.0


In [66]:
van_list = ['Richmond','Vancouver','Burnaby','Township of Langley','North Vancouver','New Westminster','Maple Ridge','Port Moody','Surrey','Langley',
            'Coquitlam','Delta','Fort Langley','White Rock','Pitt Meadows','Port Coquitlam', 'West Vancouver']

In [67]:
lib_van = lib_bc[lib_bc['CITY'].isin(van_list)]
lib_van.count()

LIBRARY_SYSTEM            66
LOCATION                  66
BRANCH_UNIQUE_ID          66
SCHOOL_DISTRICT_SERVED    66
PHONE                     65
PHYSICAL_ADDRESS          66
CITY                      66
PROVINCE                  66
POSTAL_CODE               66
LATITUDE                  66
LONGITUDE                 66
MTLS_OUTLET               66
MTLS_CIRC_B               66
CIRC_CHILD_MTLS_B         66
REF_TRANS_B               65
VISITS_B                  65
AD_INLIB_PGMS_B�          65
AD_OUT_PGMS_B�            63
ADULT_ATTEND_B�           65
CH_INLIB_PGMS_B�          65
CH_OUT_PGMS_B�            65
CHILD_ATTEND_B�           65
YA_INLIB_PGMS_B�          65
YA_OUT_PGMS_B�            64
YA_ATTEND_B�              65
ESL_INLIB_PGMS_B�         58
ESL_OUT_PGMS_B�           56
ESL_ATTEND_B�             58
LIBRARIAN_HRS_B�          63
LIB_TECH_HRS_B�           59
COMM_LIB_HRS_B�           51
OTH_HRS_B�                65
branch_copiers�           66
LEED_CERT_B�              64
SHRD_FAC�     

In [68]:
lib_van = lib_van[['BRANCH_UNIQUE_ID', 'POSTAL_CODE', 'LATITUDE', 'LONGITUDE', 'MTLS_OUTLET', 'VISITS_B', 'FLOORSPACE�', 'HRS_OPEN�', 'DAYS_OPEN�']]
lib_van.head()

,BRANCH_UNIQUE_ID,POSTAL_CODE,LATITUDE,LONGITUDE,MTLS_OUTLET,VISITS_B,FLOORSPACE�,HRS_OPEN�,DAYS_OPEN�
3,BB001,V5H 4N5,49.228301,-123.006342,"373,232","749,695","5,673.00","3,612",339.0
4,BB002,V3J 1L6,49.253814,-122.898601,"59,958","247,701",465,"2,901",341.0
5,BB003,V5C 2G6,49.282696,-123.000713,"163,916","440,899","1,860.00","3,329",341.0
6,BB004,V5E 1G8,49.218722,-122.952371,"90,645","387,327","1,627.50","3,329",341.0
25,BCOQ001,V3B 0Y1,49.281096,-122.792675,"85,271","572,820","3,205.00","3,630",357.0


In [69]:
lib_van = lib_van.reset_index(drop=True)

In [71]:
lib_van.columns = ['Branch_Id', 'Postal_Code', 'Latitude', 'Longitude', 'Total_Volumes', 'Annual_Visitors', 'Total_Space', 'Hrs_per_Year', 'Days_per_Year']
lib_van.head()

,Branch_Id,Postal_Code,Latitude,Longitude,Total_Volumes,Annual_Visitors,Total_Space,Hrs_per_Year,Days_per_Year
0,BB001,V5H 4N5,49.228301,-123.006342,"373,232","749,695","5,673.00","3,612",339.0
1,BB002,V3J 1L6,49.253814,-122.898601,"59,958","247,701",465,"2,901",341.0
2,BB003,V5C 2G6,49.282696,-123.000713,"163,916","440,899","1,860.00","3,329",341.0
3,BB004,V5E 1G8,49.218722,-122.952371,"90,645","387,327","1,627.50","3,329",341.0
4,BCOQ001,V3B 0Y1,49.281096,-122.792675,"85,271","572,820","3,205.00","3,630",357.0


In [56]:
df = library.merge(lib_van, on='Postal_Code', how='left')
df.count()

Index                  92
Latitude_x             92
Longitude_x            92
ODCAF_Facility_Type    92
Facility_Name_x        92
City                   92
Postal_Code            92
Branch_Id              10
Facility_Name_y        10
Latitude_y             10
Longitude_y            10
Total_Volumes          10
Annual_Visitors        10
Total_Space            10
Hrs_per_Year           10
Days_per_Year          10
dtype: int64

In [33]:
df = library.merge(lib_van, on='Facility_Name', how='left')
df.count()

Index                  93
Latitude               93
Longitude              93
ODCAF_Facility_Type    93
Facility_Name          93
City                   93
Postal_Code_x          93
Branch_Id               6
Postal_Code_y           6
Total_Volumes           6
Annual_Visitors         6
Total_Space             6
Hrs_per_Year            6
Days_per_Year           6
dtype: int64

In [72]:
library.Latitude = library.Latitude.round(2)
library.Longitude = library.Longitude.round(2)

lib_van.Latitude = lib_van.Latitude.round(2)
lib_van.Longitude = lib_van.Longitude.round(2)

In [82]:
df = library.merge(lib_van, on=['Latitude','Longitude'], how='left')
df.count()

Index                  95
Latitude               95
Longitude              95
ODCAF_Facility_Type    95
Facility_Name          95
City                   95
Postal_Code_x          95
Branch_Id              42
Postal_Code_y          42
Total_Volumes          42
Annual_Visitors        42
Total_Space            42
Hrs_per_Year           42
Days_per_Year          42
dtype: int64

In [83]:
df = df.dropna()
df.head()

,Index,Latitude,Longitude,ODCAF_Facility_Type,Facility_Name,City,Postal_Code_x,Branch_Id,Postal_Code_y,Total_Volumes,Annual_Visitors,Total_Space,Hrs_per_Year,Days_per_Year
5,1840,49.23,-123.01,library or archives,Bob Prittie Metrotown Library,burnaby,..,BB001,V5H 4N5,"373,232","749,695","5,673.00","3,612",339.0
7,1936,49.28,-123.07,library or archives,Britannia,vancouver,..,BVA001,V5L 4X4,"7,942","14,446",118,"1,030",206.0
9,1964,49.08,-122.67,library or archives,Brookswood Library,langley,..,BABF005,V3A 2W2,"25,669","90,898",372,"2,885",301.0
11,2090,49.25,-122.90,library or archives,Cameron,coquitlam,V3J 1L6,BB002,V3J 1L6,"59,958","247,701",465,"2,901",341.0
12,2093,49.25,-122.90,library or archives,Cameron Library,burnaby,..,BB002,V3J 1L6,"59,958","247,701",465,"2,901",341.0


In [84]:
df = df.drop('Postal_Code_x', 1)

In [86]:
df = df.replace(',','', regex=True)
df.head()

,Index,Latitude,Longitude,ODCAF_Facility_Type,Facility_Name,City,Branch_Id,Postal_Code_y,Total_Volumes,Annual_Visitors,Total_Space,Hrs_per_Year,Days_per_Year
5,1840,49.23,-123.01,library or archives,Bob Prittie Metrotown Library,burnaby,BB001,V5H 4N5,373232,749695,5673.00,3612,339.0
7,1936,49.28,-123.07,library or archives,Britannia,vancouver,BVA001,V5L 4X4,7942,14446,118,1030,206.0
9,1964,49.08,-122.67,library or archives,Brookswood Library,langley,BABF005,V3A 2W2,25669,90898,372,2885,301.0
11,2090,49.25,-122.90,library or archives,Cameron,coquitlam,BB002,V3J 1L6,59958,247701,465,2901,341.0
12,2093,49.25,-122.90,library or archives,Cameron Library,burnaby,BB002,V3J 1L6,59958,247701,465,2901,341.0


In [87]:
df[["Total_Volumes", "Annual_Visitors", "Total_Space"]] = df[["Total_Volumes", "Annual_Visitors", "Total_Space"]].apply(pd.to_numeric, errors='coerce')
df = df.reset_index(drop=True)
df.head()

,Index,Latitude,Longitude,ODCAF_Facility_Type,Facility_Name,City,Branch_Id,Postal_Code_y,Total_Volumes,Annual_Visitors,Total_Space,Hrs_per_Year,Days_per_Year
0,1840,49.23,-123.01,library or archives,Bob Prittie Metrotown Library,burnaby,BB001,V5H 4N5,373232,749695,5673.0,3612,339.0
1,1936,49.28,-123.07,library or archives,Britannia,vancouver,BVA001,V5L 4X4,7942,14446,118.0,1030,206.0
2,1964,49.08,-122.67,library or archives,Brookswood Library,langley,BABF005,V3A 2W2,25669,90898,372.0,2885,301.0
3,2090,49.25,-122.90,library or archives,Cameron,coquitlam,BB002,V3J 1L6,59958,247701,465.0,2901,341.0
4,2093,49.25,-122.90,library or archives,Cameron Library,burnaby,BB002,V3J 1L6,59958,247701,465.0,2901,341.0


### Normalize Total_Volumes, Annual_Visitors, Total_Space

In [88]:
cols_to_norm = ['Total_Volumes','Annual_Visitors', 'Total_Space']
df[cols_to_norm] = df[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df.head()

,Index,Latitude,Longitude,ODCAF_Facility_Type,Facility_Name,City,Branch_Id,Postal_Code_y,Total_Volumes,Annual_Visitors,Total_Space,Hrs_per_Year,Days_per_Year
0,1840,49.23,-123.01,library or archives,Bob Prittie Metrotown Library,burnaby,BB001,V5H 4N5,0.313325,0.387315,0.156200,3612,339.0
1,1936,49.28,-123.07,library or archives,Britannia,vancouver,BVA001,V5L 4X4,0.004510,0.006401,0.002925,1030,206.0
2,1964,49.08,-122.67,library or archives,Brookswood Library,langley,BABF005,V3A 2W2,0.019497,0.046009,0.009933,2885,301.0
3,2090,49.25,-122.90,library or archives,Cameron,coquitlam,BB002,V3J 1L6,0.048484,0.127244,0.012499,2901,341.0
4,2093,49.25,-122.90,library or archives,Cameron Library,burnaby,BB002,V3J 1L6,0.048484,0.127244,0.012499,2901,341.0


In [ ]:
# using sklearn
x = df.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled, columns=df.columns)

#### Average 3 values and obtain objective scores

In [89]:
lib_wt = df.copy()
lib_wt['Weight'] = lib_wt[['Total_Volumes','Annual_Visitors', 'Total_Space']].mean(axis=1)
lib_wt.head()

,Index,Latitude,Longitude,ODCAF_Facility_Type,Facility_Name,City,Branch_Id,Postal_Code_y,Total_Volumes,Annual_Visitors,Total_Space,Hrs_per_Year,Days_per_Year,Weight
0,1840,49.23,-123.01,library or archives,Bob Prittie Metrotown Library,burnaby,BB001,V5H 4N5,0.313325,0.387315,0.156200,3612,339.0,0.285613
1,1936,49.28,-123.07,library or archives,Britannia,vancouver,BVA001,V5L 4X4,0.004510,0.006401,0.002925,1030,206.0,0.004612
2,1964,49.08,-122.67,library or archives,Brookswood Library,langley,BABF005,V3A 2W2,0.019497,0.046009,0.009933,2885,301.0,0.025146
3,2090,49.25,-122.90,library or archives,Cameron,coquitlam,BB002,V3J 1L6,0.048484,0.127244,0.012499,2901,341.0,0.062743
4,2093,49.25,-122.90,library or archives,Cameron Library,burnaby,BB002,V3J 1L6,0.048484,0.127244,0.012499,2901,341.0,0.062743


In [90]:
lib_wt.to_csv('Library_weights.csv', index=False)